### Similitudes entre títulos de película
Este notebook calcula, para cada película del dataset, una lista de los títulos de otras películas que son muy parecidos al suyo. El diccionario de listas con esa información de similitud es utilizado por una función en el Cloud de Google para pedir una confirmación al usuario cuando el título reconocido sea muy similar a otro (y así evitar que valore una película que no es la que realmente quiere valorar).

Importamos las librerías necesarias

In [1]:
import pandas as pd
import numpy as np
import operator
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Cargamos el catálogo de películas y nos quedamos con la columna con el título de la palícula.

In [2]:
df_movies = pd.read_csv("movies_catalog_clean.csv")
titles = df_movies['title']

Calculamos las similitudes entre todos los pares de títulos posibles. Utilizamos n-gramas de longitudes 1 y 2 que, aunque suponen cierto coste en el cálculo (al considerar los pares de palabras consecutivas los vectores tienen mucha más dimensión), son muy útiles para una tarea como esta. Notemos que la aparición consecutiva de dos palabras en el título de una película es muy relevante, y si se repiten esas mismas dos palabras consecuitvas en otro título, probablemente signifique que son películas realacionadas, seguramente de la misma saga (p.ej. dos películas que lleven "star wars").

Además, fijamos un threshold de similitud muy alto para guardar el título de la película (0.7). Nuestro objetivo no es molestar al usuario con cualquier título que presenta una mínima similitud con el que detectamos, sino evitar posibles equivocaciones avisandole de películas que se llaman casi igual.

In [5]:
vectorizer = CountVectorizer(analyzer='word', stop_words="english", ngram_range=(1, 2))
count_vectorized = vectorizer.fit_transform(titles)
similarities = cosine_similarity(count_vectorized, count_vectorized)
similarities_json = {}
for i in range(similarities.shape[0]):
    lista = list(enumerate(similarities[i]))
    lista.pop(i)
    lista = list(filter(lambda l : l[1] > 0.7, lista))
    lista = sorted(lista, key=lambda x: x[1], reverse=True)
    lista = lista[:5]
    similarities_json[titles[i]] = list(map(lambda item : titles[item[0]], lista))
    # Mostramos la lista de similitudes cada 100 películas
    if i % 100 == 0:
        print(i, similarities_json[titles[i]]) 

0 ['Toy Story 2', 'Toy Story 3', 'Toy Story of Terror!']
100 []
200 []
300 []
400 []
500 []
600 []
700 []
800 []
900 []
1000 ['The Air Up There', 'Up in the Air', 'Something in the Air', 'Air', 'On Air']
1100 []
1200 ['Grand Hotel Excelsior']
1300 []
1400 []
1500 []
1600 []
1700 ['Monster']
1800 []
1900 []
2000 []
2100 []
2200 []
2300 []
2400 []
2500 []
2600 []
2700 []
2800 []
2900 []
3000 []
3100 []
3200 []
3300 []
3400 []
3500 []
3600 []
3700 []
3800 ['Friday', 'Next Friday']
3900 []
4000 []
4100 []
4200 []
4300 []
4400 []
4500 []
4600 []
4700 []
4800 []
4900 []
5000 []
5100 []
5200 []
5300 []
5400 []
5500 []
5600 []
5700 ['Bring It On', 'Bring It On: All or Nothing']
5800 []
5900 []
6000 []
6100 []
6200 []
6300 []
6400 ['Ice Age']
6500 ['The Lost City of Z']
6600 []
6700 []
6800 ['X: The Unknown', 'The Unknown']
6900 []
7000 []
7100 []
7200 []
7300 []
7400 []
7500 ['Mirrors 2']
7600 []
7700 []
7800 []
7900 []
8000 []
8100 []
8200 []
8300 []
8400 []
8500 ['Mr. Six']
8600 []
8700 ['In

Guardamos el diccionario con las listas anteriores de título similares en el fichero externo "movie_name_similarity.json"

In [6]:
with open('./movie_name_similarity.json', 'w') as fp:
    json.dump(similarities_json, fp)

### Ejemplos de sugerencias

Hacemos una prueba de cómo se mostraría títulos similares a "The Jungle Book" y "The Hunger Games" a partir del diccionario anterior.

In [11]:
with open('movie_name_similarity.json', 'r') as f:
    similarities = json.load(f)
    
def sugerencia(value):
    response_text = ""
    if similarities[value] == []:
        response_text = "I understand that you want to rate the film '{}'. I'm right?".format(value)
    else:
        response_text = "I think you want to rate the movie '{}'. However, I've found these other films that are similarly named:\n\n".format(value)
        for film_name in similarities[value]:
            response_text += "'{}'\n".format(film_name)
        response_text += '\nI am right or you were referring to another film?'
    print(response_text)

sugerencia("The Jungle Book")
print("=" * 100)
sugerencia("The Hunger Games")

I think you want to rate the movie 'The Jungle Book'. However, I've found these other films that are similarly named:

'The Jungle Book 2'
'Jungle Book'

I am right or you were referring to another film?
I think you want to rate the movie 'The Hunger Games'. However, I've found these other films that are similarly named:

'The Hunger Games: Catching Fire'
'The Hunger Games: Mockingjay - Part 1'
'The Hunger Games: Mockingjay - Part 2'

I am right or you were referring to another film?
